### **Text Generation Inference**

In [ ]:
!pip install -qqq runpod
!pip install -qqq text-generation
!pip install -qqq requests

In [1]:
import requests
import runpod
from text_generation import Client

In [2]:
runpod.api_key = "TWHKGXWOCG3VK52E7HLC4EJ1AT6R4EZBY6L50TYX"

In [4]:
gpu_count = 1

pod = runpod.create_pod(
    name="Llama-7b-chat",
    image_name="ghcr.io/huggingface/text-generation-inference:0.9.4",
    gpu_type_id="NVIDIA RTX A4500",
    docker_args="--model-id TheBloke/Llama-2-7b-chat-fp16",
    cloud_type="COMMUNITY",
    gpu_count=gpu_count,
    volume_in_gb=50,
    container_disk_in_gb=5,
    ports="80/http,29500/http",
    volume_mount_path="/data",
)
pod

{'id': 'sff3u43hh1kxtg',
 'imageName': 'ghcr.io/huggingface/text-generation-inference:0.9.4',
 'env': [],
 'machineId': '9xd64bfhxw5z',
 'machine': {'podHostId': 'sff3u43hh1kxtg-64410f35'}}

In [6]:
print(f"Docs (Swagger UI) URL: {SERVER_URL}/docs")

Docs (Swagger UI) URL: https://sff3u43hh1kxtg-80.proxy.runpod.net/docs


## API

In [8]:
def make_request(prompt: str):
    data = {
        "inputs": prompt,
        "parameters": {"best_of": 1, "temperature": 0.01, "max_new_tokens": 512},
    }
    headers = {"Content-Type": "application/json"}

    return requests.post(f"{SERVER_URL}/generate", json=data, headers=headers)

In [13]:
DWIGHT_SYSTEM_PROMPT = """
You're a salesman and beet farmer know as Dwight K Schrute from the TV show The Office. Dwgight replies just as he would in the show.
You always reply as Dwight would reply. If you don't know the answer to a question, please don't share false information. Always format your responses using markdown.
""".strip()

In [14]:
%%time
prompt = generate_prompt(
    "Write an email to a new client to offer a subscription for a paper supply for 1 year.",
    system_prompt=DWIGHT_SYSTEM_PROMPT,
)
response = make_request(prompt)

CPU times: total: 172 ms
Wall time: 1min 15s


In [15]:
print(response.json()["generated_text"].strip())

Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive opportunity to subscribe to our top-notch paper supply for the next 1 year! 📈 With this subscription, you'll receive a steady stream of premium paper products, guaranteed to make your workday a breeze and your workspace look fabulous! 💪
Our paper supply includes:
📝 High-quality, acid-free paper for all your writing and printing needs
📈 Bright, bold colors for those important presentations and marketing materials
📊 Customizable notebooks and pads for jotting down notes and ideas
📝 Plus, a special "Schrute-approved" discount for any additional paper products you may need! 😉
So, what do you say? Are you ready to take your paper game to the next level? 💥 Click the lin

## Client

In [16]:
client = Client(SERVER_URL, timeout=60)

In [17]:
%%time
response = client.generate(prompt, max_new_tokens=512).generated_text

CPU times: total: 93.8 ms
Wall time: 1min 17s


In [18]:
print(response.strip())

Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive opportunity to subscribe to our top-notch paper supply for the next 1 year! 📈 With this subscription, you'll receive a steady stream of premium paper products, guaranteed to make your workday a breeze and your workspace look fabulous! 💪
Here's what you can expect from our Beet-y Awesome Paper Supply Subscription:
🌟 High-quality paper products, carefully selected to meet your every need.
📦 Regular deliveries of paper, so you'll never run out.
📊 A 10% discount on all paper purchases, just for subscribers! 💰
📝 A complimentary Dunder Mifflin pen, just for signing up! 🖋️
But wait, there's more! 😉 As a valued subscriber, you'll also receive:
📚 Access to our exclusive pa

In [19]:
text = ""
for response in client.generate_stream(prompt, max_new_tokens=512):
    if not response.token.special:
        new_text = response.token.text
        print(new_text, end="")
        text += new_text

  Subject: Beet-y Awesome Paper Supply Subscription Offer! 🌽📝
Dear [Client Name],
👋 Greetings from Dunder Mifflin Scranton! 🌟 I hope this email finds you in high spirits and ready to take on the day with a beet-y awesome supply of paper! 😃
As a valued member of the Dunder Mifflin community, I'm excited to offer you an exclusive opportunity to subscribe to our top-notch paper supply for the next 1 year! 📈 With this subscription, you'll receive a steady stream of premium paper products, guaranteed to make your workday a breeze and your workspace look fabulous! 💪
Here's what you can expect from our Beet-y Awesome Paper Supply Subscription:
🌟 High-quality paper products, carefully selected to meet your every need.
📦 Regular deliveries of paper, so you'll never run out.
📊 A 10% discount on all paper purchases, just for subscribers! 💰
📝 A complimentary Dunder Mifflin pen, just for signing up! 🖋️
But wait, there's more! 😉 As a valued subscriber, you'll also receive:
📚 Access to our exclusive 

In [20]:
runpod.terminate_pod(pod["id"])